In [1]:
import ast
import json
import pandas as pd
from pathlib import Path
import shutil
from PIL import Image
from jinja2 import Environment, FileSystemLoader


In [2]:
# Constants
CSV_DIR_PATH = Path(".")
GROUP_DATA_DIR = Path("../../group-data")
HOSTING_PATH = GROUP_DATA_DIR.parent / "kerzendorf-lab.github.io"
TEMPLATE_DIR_PATH = GROUP_DATA_DIR.parent / "groupwebsite_generator" / "templates"
WEBSITE_DATA_PATH = GROUP_DATA_DIR / "website_data/"
GALLERY_CONTENT_SOURCE = WEBSITE_DATA_PATH / "content" / "gallery"
SOURCE_ASSETS = GROUP_DATA_DIR.parent / "groupwebsite_generator" / "assets"
SUB_RESEARCH_PATH = HOSTING_PATH / "sub_research"
OPPORTUNITIES_PATH = WEBSITE_DATA_PATH / "content" / "opportunities.json"

TAG_COLORS = {
    'paper': '#FF6B6B',
    'poster': '#4ECDC4',
    'talk': '#45B7D1',
    'award': '#96CEB4',
    'new team member': '#FFBE0B',
    'phd': '#9B5DE5',
    'conference': '#FF006E',
    'undergraduate': '#8338EC',
    'event': '#3A86FF',
    'achievement': '#FB5607',
    'astrophysics': '#2EC4B6',
    'machine learning': '#FF9F1C',
    'software': '#E71D36',
    'research': '#011627',
    'news': '#41EAD4'
}

# Setup Jinja2 environment

In [3]:
# Setup Jinja2 environment
environment = Environment(
    loader=FileSystemLoader(TEMPLATE_DIR_PATH), extensions=["jinja2.ext.loopcontrols", "jinja2.ext.do"]
)

# Helper Functions
def page_link(a):
    """Return the HTML file name after replacing blank spaces with underscores"""
    return a.replace(" ", "_") if " " in a else a
def get_tag_color(tag):
    """Get color for a specific tag, with fallback to default"""
    return TAG_COLORS.get(tag.lower(), '#6c757d')

environment.globals["page_link"] = page_link
environment.globals['tag_colors'] = TAG_COLORS
environment.globals['get_tag_color'] = get_tag_color

def create_page(template, html, **kwargs):
    """Create an HTML page using a Jinja2 template and save it to a specified path"""
    page_template = environment.get_template(template)
    template_level = html.count("/")
    page_html_path = HOSTING_PATH / html
    page_html_path.parent.mkdir(parents=True, exist_ok=True)
    page_content = page_template.render(TEMPLATE_LEVEL=template_level, **kwargs)
    with open(page_html_path, mode="w", encoding="utf-8") as page:
        page.write(page_content)

# Read Data

In [4]:
# Load member and article data from CSVs
members_df = pd.read_csv(CSV_DIR_PATH / "members.csv", index_col=0)
education_df = pd.read_csv(CSV_DIR_PATH / "education.csv", index_col=0)
experiences_df = pd.read_csv(CSV_DIR_PATH / "experiences.csv", index_col=0)
projects_df = pd.read_csv(CSV_DIR_PATH / "projects.csv", index_col=0)
awards_df = pd.read_csv(CSV_DIR_PATH / "awards.csv", index_col=0)
outreach_df = pd.read_csv(CSV_DIR_PATH / "outreach.csv", index_col=0)
documents_df = pd.read_csv(CSV_DIR_PATH / "documents.csv", index_col=0)

current_members_with_info = pd.read_csv(CSV_DIR_PATH / "current_members.csv", index_col=0)
# Replace NaN with empty string for current_project_title to avoid displaying "nan"
current_members_with_info['current_project_title'] = current_members_with_info['current_project_title'].fillna('')
alumni_members_with_info = pd.read_csv(CSV_DIR_PATH / "alumni_members.csv", index_col=0)
articles_df = pd.read_csv(CSV_DIR_PATH / "articles.csv", index_col=0, parse_dates=['date'])
news_df = pd.read_csv(CSV_DIR_PATH / "news.csv", index_col=0, parse_dates=['date'])
research_df = pd.read_csv(CSV_DIR_PATH / "research.csv", index_col=0, parse_dates=['date'])



In [5]:
# Parse dates for dataframes with date columns
for df in [education_df, experiences_df, projects_df, outreach_df]:
    if 'start_date' in df.columns:
        df['start_date'] = pd.to_datetime(df['start_date'])
    if 'end_date' in df.columns:
        df['end_date'] = pd.to_datetime(df['end_date'])

# Awards has additional 'date' column
if 'date' in awards_df.columns:
    awards_df['date'] = pd.to_datetime(awards_df['date'])
if 'start_date' in awards_df.columns:
    awards_df['start_date'] = pd.to_datetime(awards_df['start_date'])
if 'end_date' in awards_df.columns:
    awards_df['end_date'] = pd.to_datetime(awards_df['end_date'])

In [6]:
# Convert string columns back to their original types
for df in [articles_df, news_df, research_df]:
    df['content'] = df['content'].apply(ast.literal_eval)
    df['links'] = df['links'].apply(ast.literal_eval)


In [7]:
# Group dataframes
def group_df(df):
    """Group dataframe by index and convert to nested dict format for templates"""
    return df.fillna("").groupby(level=0).apply(lambda x: x.to_dict('records')).to_frame('info').to_dict('index')


education = group_df(education_df)
experience = group_df(experiences_df)
projects = group_df(projects_df)
awards = group_df(awards_df)
outreach = group_df(outreach_df)
documents = group_df(documents_df)

In [8]:
# Setup socials and dicts
# Load social fields from schema
social_schema_path = GROUP_DATA_DIR / "schemas/members/social_links.json"
social_schema = json.loads(social_schema_path.read_text())
social_cols = list(social_schema['properties'].keys())
socials = members_df[social_cols].fillna('').to_dict('index')

In [9]:
all_members_dict = members_df.to_dict("index")
all_articles_dict = {
    aid: {**data, 'article_id': aid}
    for aid, data in articles_df.to_dict("index").items()
}


In [10]:
general = json.loads((WEBSITE_DATA_PATH / "general.json").read_text())

for person_id, person_data in members_df.iterrows():
    create_page(
        "individual_person.html.j2",
        f"members/{person_id}/{person_id}.html",
        general=general,
        member_id=person_id,
        member_data=person_data,
        socials=socials,
        documents=documents,
        education=education,
        experience=experience,
        projects=projects,
        awards=awards,
        outreach=outreach,
        content=all_articles_dict,
    )
print(f"Created {len(members_df)} individual member pages")


Created 34 individual member pages


In [11]:
# Copy assets and load JSON files
shutil.copytree(SOURCE_ASSETS, HOSTING_PATH / "assets", dirs_exist_ok=True)

homepage = json.loads((WEBSITE_DATA_PATH / "homepage.json").read_text())
contact = json.loads((WEBSITE_DATA_PATH / "contact.json").read_text())
support = json.loads((WEBSITE_DATA_PATH / "support.json").read_text())
research = json.loads((WEBSITE_DATA_PATH / "research_categories.json").read_text())

# Get recent content for homepage
recent_content_df = articles_df.sort_values(
    ["category", "date"], ascending=[True, False]
).groupby("category").head(1)


# Create homepage
create_page(
    "homepage.html.j2",
    "index.html",
    general=general,
    homepage=homepage,
    recent_content=recent_content_df.reset_index().to_dict(orient="records"),
)


In [12]:

# Create current members page
create_page(
    "current_members.html.j2",
    "current_members.html",
    general=general,
    current_members=current_members_with_info,
    socials=socials
)


In [13]:
# Create alumni page
create_page(
    "alumni_members.html.j2",
    "alumni_members.html",
    general=general,
    alumni_members=alumni_members_with_info,
)

In [14]:
# Create contact page
create_page(
    "contact.html.j2",
    "Contact.html",
    general=general,
    contact=contact
)

In [15]:

# Create support page
create_page(
    "support.html.j2",
    "Support.html",
    general=general,
    support=support
)


In [16]:
# Create research page
create_page(
    "research.html.j2",
    "Research.html",
    general=general,
    content=research_df.reset_index(),
    research=research,
    current_members=all_members_dict,
)

In [17]:
# Create sub_research directory
SUB_RESEARCH_PATH.mkdir(parents=True, exist_ok=True)

In [18]:

# Create category pages
for category in research_df["category"].unique():
    create_page(
        "sub_research_frontpage.html.j2",
        f"sub_research/{page_link(category.lower())}.html",
        general=general,
        research=research,
        content=research_df.reset_index(),
        category=category,
        current_members=all_members_dict,
    )


In [19]:

# Create individual research pages
for article_id, ind_research_values in research_df.iterrows():
    destination_research_path = f"sub_research/{page_link(ind_research_values.category.lower())}/{page_link(article_id.lower())}.html"
    if ind_research_values['category'] == "Software":
        destination_research_path = f"sub_research/{page_link(article_id.lower())}.html"

    folder_path = SUB_RESEARCH_PATH / page_link(ind_research_values.category.lower())
    folder_path.mkdir(parents=True, exist_ok=True)
    create_page(
        "research_page_no_twitter.html.j2",
        destination_research_path,
        general=general,
        content=ind_research_values,
        member_data=all_members_dict,
        article_id=article_id,
    )


In [20]:

# Create news page
create_page(
    "news.html.j2",
    "News.html",
    general=general,
    content=news_df.reset_index(),
    category="News",
    member_data=all_members_dict,
)

# Create individual news pages
news_dict_list = news_df.reset_index().to_dict('records')
for news_item in news_dict_list:
    create_page(
        "news_page_no_twitter.html.j2",
        f"news/{page_link(news_item['article_id'].lower())}.html",
        general=general,
        content=news_item,
        member_data=all_members_dict,
        category="News"
    )


In [21]:
# Create join us page
with open(OPPORTUNITIES_PATH, 'r') as f_opp:
    opportunities = json.load(f_opp)

create_page(
    "join_us.html.j2",
    "Join_Us.html",
    general=general,
    opportunities=opportunities
)